In [235]:
import os
import shutil
from threading import Thread
from glob import glob
from subprocess import Popen

import pandas as pd
import numpy as np

import datetime
import time

from features import get_features, perform_backtest

from twitter_stream import twitter_stream
from price_stream import price_stream
from utils.get_price_data import get_price
from utils.common_utils import get_root_dir, merge_csvs
from process import processor, get_sentiment, clean_further, add_keyword, clean_profile

In [236]:
dir = get_root_dir()

In [237]:
keywords = pd.read_csv('keywords.csv')

In [238]:
# files = glob('data/test/*')
files = glob('data/twitter_stream/*')

In [239]:
for idx, file in enumerate(files):
    old_name = os.path.join(dir, files[idx])
    files[idx] = os.path.join(dir, files[idx].replace('twitter_stream/', 'temp/'))
    shutil.move(old_name, files[idx])
    print('moving {}'.format(file))

moving data/twitter_stream/2019-08-10.csv


In [240]:
print(files)

['/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/temp/2019-08-10.csv']


In [241]:
combined = merge_csvs(files)

In [242]:
df = pd.read_csv(combined)

In [243]:
df, user_info = processor(df)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:101: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [244]:
print('first processed')

first processed


In [245]:
len(df)

720418

In [246]:
len(user_info)

333113

In [247]:
df['Time'].dtype

dtype('int64')

In [248]:
df['ID'].dtype

dtype('int64')

In [249]:
savefile = os.path.join(dir, 'data/all_cleaned.csv')
profilefile = os.path.join(dir, 'data/cleaned_profile.csv')

In [250]:
for file in files:
    os.remove(file)

In [251]:
print("getting sentiment and all")

getting sentiment and all


In [252]:
df['ID'] = df['ID'].astype(int)

In [253]:
df = add_keyword(df)

In [254]:
df['keyword'].value_counts(dropna=False)

invalid    473180
BTC        169180
ETH         28680
TRX         14456
XRP         10900
LTC          6781
EOS          5157
MIOTA        3308
LINK         2030
BCHABC       1216
XLM          1121
ADA           718
XMR           693
ZEC           588
XTZ           479
BNB           475
BSV           447
DASH          326
NEO           318
ETC           205
BTG            79
ZRX            43
OMG            19
ETP            12
SAN             7
Name: keyword, dtype: int64

In [255]:
df = df[df['keyword'] != 'invalid']

In [256]:
df = df.reset_index(drop=True)

In [257]:
df = get_sentiment(df)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:199: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



java -jar /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/utils/SentiStrength.jar sentidata /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/utils/SentiStrength_Data/ input /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/temp/tweets


In [258]:
df['pos_neg'].value_counts(dropna=False)

 0    158500
 1     36661
 2     20439
-1     17245
-2      5688
 3      4913
-3      1876
 4      1834
-4        82
Name: pos_neg, dtype: int64

In [259]:
savefile

'/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/all_cleaned.csv'

In [260]:
df.to_csv(savefile, index=None)

In [261]:
len(df)

247238

In [265]:
df = pd.read_csv(savefile)

In [263]:
df = clean_further(df)

In [264]:
df.to_csv(savefile, index=None)

In [266]:
profilefile

'/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/cleaned_profile.csv'

In [267]:
if os.path.isfile(profilefile):
    user_info = pd.concat([user_info, pd.read_csv(profilefile)])
    user_info = clean_profile(user_info)

In [268]:
len(user_info)

443808

In [269]:
user_info.to_csv(profilefile, index=None)

In [270]:
df = df.sort_values('Time')

In [271]:
def save_to_file(df):
    global dir
    keyword = df.iloc[0]['keyword']
    print(keyword)
    
    df = df.drop('keyword', axis=1)

    coinwise_folder = os.path.join(dir, "data/coinwise")

    if not os.path.isdir(coinwise_folder):
        os.makedirs(coinwise_folder)

    savefile = "{}/{}.csv".format(coinwise_folder, keyword)
    df.to_csv(savefile, index=None)

In [272]:
df.groupby('keyword').apply(save_to_file)

ADA
ADA
BCHABC
BNB
BSV
BTC
BTG
DASH
EOS
ETC
ETH
ETP
LINK
LTC
MIOTA
NEO
OMG
SAN
TRX
XLM
XMR
XRP
XTZ
ZEC
ZRX


""


In [273]:
curr_start = df.iloc[0]['Time']

In [274]:
curr_end = df.iloc[-1]['Time']

In [275]:
curr_start

'2019-08-09 23:59:55'

In [276]:
curr_end

'2019-08-10 23:59:54'

In [277]:
storage_dir = os.path.join(dir, 'data/storage/all_cleaned/')

In [278]:
storage_dir

'/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/storage/all_cleaned/'

In [279]:
if not os.path.isdir(storage_dir):
    os.makedirs(storage_dir)

In [280]:
storagename = os.path.join(storage_dir, 'all_cleaned_{}.csv'.format(int(time.time())))

In [281]:
storagename

'/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/storage/all_cleaned/all_cleaned_1567304647.csv'

In [282]:
shutil.move(savefile, storagename)

'/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/storage/all_cleaned/all_cleaned_1567304647.csv'

In [283]:
keywords[:2]

,Symbol,exchange_name,pair_name,Currency,Keywords,Coinmarketcap,marketcap,Marketcap_Tier,Tweets_Tier,Blacklisted,Shortable
0,SAN,Bitfinex,SAN/USD,Santiment Network Token,"Santiment Network Token, $SAN",https://coinmarketcap.com/currencies/santiment/,33204098,7,7,1,1
1,XMR,Bitfinex,XMR/USD,Monero,"Monero, XMR",https://coinmarketcap.com/currencies/monero/,1451170205,3,3,0,1


In [284]:
##FEATURES CALCULATON##

#PREPROCESS

In [285]:
symbol = 'XMR'

In [286]:
tweet_df = pd.read_csv(dir + '/data/coinwise/{}.csv'.format(symbol))

In [287]:
tweet_df['Time'] = pd.to_datetime(tweet_df['Time'])

In [289]:
price_df = get_price(symbol)

In [290]:
tweet_df = tweet_df.sort_values('Time')

In [291]:
tweet_df = tweet_df.set_index('Time')

In [292]:
tweet_df.index = tweet_df.index.ceil(freq='30Min') 

In [293]:
tweet_df = tweet_df.reset_index()

In [294]:
#FEATURES FUNCTION

In [295]:
features_dir = get_root_dir() + "/data/features"
features_file = features_dir + '/{}.csv'.format(symbol)

In [296]:
userwise_inf_file = os.path.join(get_root_dir(), 'data/userwise_influence.csv')

In [297]:
userwise_inf = pd.read_csv(userwise_inf_file)

In [298]:
tweet_df['Time'] = pd.to_datetime(tweet_df['Time'])
tweet_df = tweet_df.sort_values('Time')

In [299]:
tweet_df = tweet_df.merge(userwise_inf[['username', 'avg_influence', 'total_influence']], left_on='User', right_on='username', how='left')
tweet_df['avg_influence'] = tweet_df['avg_influence'].fillna(2) #half the average if that user does not exist. This number goes down as our dataset goes up
tweet_df['total_influence'] = tweet_df['total_influence'].fillna(6)

In [300]:
price_df = price_df[(price_df['Time'] >= curr_start) & (price_df['Time'] <= curr_end)].reset_index(drop=True)

In [301]:
from features import tweets_to_features

In [302]:
features = tweet_df.groupby('Time').apply(tweets_to_features)

In [303]:
features = price_df.merge(features, how='left', on='Time')
features = features.fillna(0)

In [311]:
features

,Time,Open,High,Low,Close,Volume,number_of_tweets,total_influence,sentistrength_total,sentistrength_total_mean
0,2019-08-10 00:00:00,93.631,94.208,93.631,94.208,1949.008901,0.0,0.000000,0.000000,0.000000
1,2019-08-10 00:30:00,94.103,94.679,94.022,94.605,3479.600715,11.0,23.000000,8.000000,0.347826
2,2019-08-10 01:00:00,94.605,94.605,94.239,94.454,3286.455260,9.0,18.553106,4.553106,0.245409
3,2019-08-10 01:30:00,94.484,94.484,94.111,94.165,1120.518932,13.0,24.027508,3.025079,0.125901
4,2019-08-10 02:00:00,94.212,94.335,93.484,93.484,8271.883761,14.0,58.360669,34.360669,0.588764
5,2019-08-10 02:30:00,94.085,94.085,93.260,93.933,4787.934800,19.0,114.968343,26.106213,0.227073
6,2019-08-10 03:00:00,93.932,93.933,93.387,93.449,1004.024432,9.0,18.838702,7.384189,0.391969
7,2019-08-10 03:30:00,93.449,93.803,93.449,93.620,172.846285,10.0,20.471562,4.000000,0.195393
8,2019-08-10 04:00:00,93.704,93.864,93.503,93.864,25.454548,5.0,10.000000,2.000000,0.200000
9,2019-08-10 04:30:00,93.864,93.864,93.627,93.674,2819.582700,10.0,22.366116,-0.633884,-0.028341


In [ ]:
#CHECK EXISTANCE

In [319]:
features = pd.concat([pd.read_csv(features_file), features])

In [323]:
features['Time'] = pd.to_datetime(features['Time'])

In [324]:
features = features.sort_values('Time')
features = features.drop_duplicates('Time',keep='last')

In [329]:
test = features['Time'] - features.shift(1)['Time']

In [336]:
test[test != pd.Timedelta('0 days 00:30:00')]

0   NaT
Name: Time, dtype: timedelta64[ns]

In [ ]:
#DONE

In [337]:
features.to_csv(features_file, index=None)

In [146]:
#test this without condition later

In [ ]:
#combining old_features and new is wrong

In [147]:
features = get_features(tweet_df, price_df, symbol, curr_start, curr_end)

In [149]:
features.sort_values('Time')

,Time,Open,High,Low,Close,Volume,number_of_tweets,total_influence,sentistrength_total,sentistrength_total_mean
0,2019-08-12 00:00:00,92.448,93.000,92.448,92.783,19857.440442,0.0,0.000000,0.000000,0.000000
1,2019-08-12 00:30:00,93.684,93.734,92.926,93.027,97064.779840,17.0,44.087618,-2.000000,-0.045364
2,2019-08-12 01:00:00,93.027,93.307,92.906,93.307,6042.780551,15.0,29.236571,3.068785,0.104964
3,2019-08-12 01:30:00,91.742,92.009,91.742,92.009,1759.981185,15.0,31.002922,21.441464,0.691595
4,2019-08-12 02:00:00,91.972,92.009,91.832,91.832,193.191600,15.0,66.677350,-32.599687,-0.488917
5,2019-08-12 02:30:00,93.565,94.434,93.353,93.998,20459.612104,16.0,42.361527,26.914633,0.635356
6,2019-08-12 03:00:00,91.627,91.627,91.500,91.500,471.608200,11.0,22.787703,2.000000,0.087767
7,2019-08-12 03:30:00,93.191,93.193,92.153,92.500,16677.645860,11.0,21.000000,-4.000000,-0.190476
8,2019-08-12 04:00:00,92.636,92.636,92.275,92.293,3747.382233,9.0,16.201245,0.180296,0.011129
9,2019-08-12 04:30:00,91.408,91.408,91.339,91.360,500.562740,11.0,23.000000,2.000000,0.086957


In [338]:
perform_backtest(symbol, n_fast_par=24, n_slow_par=52, long_macd_threshold_par=2, long_per_threshold_par=1, 
                        long_close_threshold_par=1, short_macd_threshold_par=-1, short_per_threshold_par=-1, short_close_threshold_par=0.5,
                        initial_cash=10000, comission=0.1)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/features.py:277: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



In [20]:
df = df.sort_values('Time')
df = df.set_index('Time')
df.index = df.index.ceil(freq='30Min')  
df = df.reset_index()

In [22]:
## VERIFYING TIME

In [23]:
pd.Timestamp('2019-08-09 06:01:00').ceil(freq='30Min') #test of ceil

Timestamp('2019-08-09 06:30:00')

In [24]:
price_df = pd.read_csv('data/price/{}.csv'.format(coin_name))

In [25]:
price_df['Time'] = pd.to_datetime(price_df['Time'])

In [26]:
price_df.head(20)

,Time,Open,High,Low,Close,Volume
0,2019-07-01 00:00:00,10854.10,10883.19,10840.10,10876.07,1.496492e+06
1,2019-07-01 00:01:00,10877.24,10885.57,10853.20,10881.44,1.099121e+06
2,2019-07-01 00:02:00,10880.16,10880.17,10835.00,10835.00,1.574824e+06
3,2019-07-01 00:03:00,10835.02,10840.00,10811.99,10828.56,1.719258e+06
4,2019-07-01 00:04:00,10826.60,10827.97,10775.00,10812.51,1.394439e+06
5,2019-07-01 00:05:00,10812.51,10826.74,10736.05,10736.05,2.634270e+06
6,2019-07-01 00:06:00,10735.78,10787.36,10719.00,10778.28,2.702564e+06
7,2019-07-01 00:07:00,10776.06,10840.00,10770.59,10830.20,1.606807e+06
8,2019-07-01 00:08:00,10825.82,10850.01,10805.70,10831.59,1.361710e+06
9,2019-07-01 00:09:00,10829.84,10870.00,10829.73,10862.75,1.045778e+06


In [27]:
get_price('BTC', duration='10Min').head(2) #uses the next 10 mins. Manually verifying that

,Time,Open,High,Low,Close,Volume
0,2019-07-01 00:10:00,10854.10,10885.57,10719.00,10862.75,1.663526e+07
1,2019-07-01 00:20:00,10858.68,11047.01,10812.17,10998.14,1.623573e+07


In [28]:
## VERIFIED TIME ##

In [29]:
price_df = get_price('BTC')
tweet_df = df[df['keyword'] == coin_name].drop('keyword', axis=1).reset_index(drop=True)

curr_start = df.iloc[0]['Time']
curr_end = df.iloc[-1]['Time']

In [30]:
features = get_features(tweet_df, price_df, coin_name, curr_start, curr_end) #verified

In [31]:
#Now verify cerebro

In [32]:
from features import perform_backtest

In [33]:
perform_backtest(coin_name, n_fast_par=24, n_slow_par=52, long_macd_threshold_par=2, long_per_threshold_par=1, 
                        long_close_threshold_par=1, short_macd_threshold_par=-1, short_per_threshold_par=-1, short_close_threshold_par=0.5,
                        initial_cash=10000, comission=0.1, df=features)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/features.py:277: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



In [ ]:
os.remove('data/features/BTC.csv')